In [3]:
import pandas as pd
import numpy as np


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics

In [5]:
matches_data = pd.read_csv('/Users/prathyushachelemella/Desktop/games.csv')

columns_to_drop = ['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'TEAM_ID_home', 'FG_PCT_home', 'FT_PCT_home', 
                   'FG3_PCT_home', 'TEAM_ID_away','FG_PCT_away', 'FT_PCT_away','FG3_PCT_away']

matches_data = matches_data.drop(columns_to_drop, axis=1)
matches_data.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,AST_home,REB_home,PTS_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,1610612740,1610612759,2022,126.0,25.0,46.0,117.0,23.0,44.0,1
1,22200478,1610612762,1610612764,2022,120.0,16.0,40.0,112.0,20.0,37.0,1
2,22200466,1610612739,1610612749,2022,114.0,22.0,37.0,106.0,20.0,46.0,1
3,22200467,1610612755,1610612765,2022,113.0,27.0,49.0,93.0,15.0,46.0,1
4,22200468,1610612737,1610612741,2022,108.0,22.0,47.0,110.0,20.0,47.0,0


In [6]:
matches_data.shape

(26651, 11)

In [7]:
matches_data.columns

Index(['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'PTS_home',
       'AST_home', 'REB_home', 'PTS_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS'],
      dtype='object')

In [8]:
win_df=matches_data.groupby(['GAME_ID','SEASON']).sum()['HOME_TEAM_WINS'].reset_index()

win_df

,GAME_ID,SEASON,HOME_TEAM_WINS
0,10300001,2003,1
1,10300002,2003,1
2,10300003,2003,0
3,10300004,2003,1
4,10300005,2003,1
...,...,...,...
26617,52100111,2021,1
26618,52100121,2021,1
26619,52100131,2021,1
26620,52100201,2021,0


In [8]:
win_df = win_df[win_df['HOME_TEAM_WINS']==1]
win_df['SEASON'] = win_df['SEASON'].apply(lambda x:x+1)#to get target
win_df

<ipython-input-8-2ee73ae2e93d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  win_df['SEASON'] = win_df['SEASON'].apply(lambda x:x+1)#to get target


,GAME_ID,SEASON,HOME_TEAM_WINS
0,10300001,2004,1
1,10300002,2004,1
3,10300004,2004,1
4,10300005,2004,1
6,10300007,2004,1
...,...,...,...
26614,52000201,2021,1
26616,52100101,2022,1
26617,52100111,2022,1
26618,52100121,2022,1


In [11]:
data = win_df.copy()

from sklearn.model_selection import train_test_split
X = matches_data.drop(['HOME_TEAM_WINS'], axis=1)
y = matches_data['HOME_TEAM_WINS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train.shape,X_test.shape
X_train.columns

Index(['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON', 'PTS_home',
       'AST_home', 'REB_home', 'PTS_away', 'AST_away', 'REB_away'],
      dtype='object')

In [12]:
cf = ColumnTransformer(transformers = [
    ('tnf1',OneHotEncoder(sparse=False,drop='first'),['HOME_TEAM_ID','VISITOR_TEAM_ID','SEASON'])
],remainder='passthrough')

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Create a pipeline with a SimpleImputer and a LogisticRegression classifier
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', LogisticRegression())
])

# Fit the pipeline to your data
pipeline.fit(X_train, y_train)

# Make predictions using the trained pipeline
y_pred = pipeline.predict(X_test)
print(metrics.accuracy_score(y_test,y_pred))

pipeline.predict_proba(X_test)[10]

0.5871318701932096


array([0.41364844, 0.58635156])

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


pipeline2 = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', RandomForestClassifier())
])

pipeline2.fit(X_train,y_train)
y_pred = pipeline2.predict(X_test)
print(metrics.accuracy_score(y_test,pipeline2.predict(X_test)))

pipeline2.predict_proba(X_test)[10]



0.996060776589758


array([0.99, 0.01])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


df = pd.read_csv("/Users/prathyushachelemella/Desktop/games.csv")


df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'SEASON', 'TEAM_ID_home', 'TEAM_ID_away'], axis=1, inplace=True)


df['PTS_DIFF'] = df['PTS_home'] - df['PTS_away']


df['ELO_HOME'] = 1500
df.loc[1:, 'ELO_HOME'] = np.cumsum((df['HOME_TEAM_WINS'] - 0.5) * 20)


df['ELO_AWAY'] = 1500
df.loc[1:, 'ELO_AWAY'] = np.cumsum((1 - df['HOME_TEAM_WINS'] - 0.5) * 20)


df.dropna(inplace=True)


plt.scatter(df['PTS_DIFF'], df['FG_PCT_home'] - df['FG_PCT_away'])
plt.xlabel('Points Difference')
plt.ylabel('Field Goal Percentage Difference')


X = df.drop(['HOME_TEAM_WINS'], axis=1)
y = df['HOME_TEAM_WINS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
svm_y_pred = svm.predict(X_test)


lr = LinearRegression()
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)


xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)


rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)
rfc_y_pred = rfc.predict(X_test)


print('SVM Accuracy:', accuracy_score(y_test, svm_y_pred))
print('Linear Regression Accuracy:', accuracy_score(y_test, np.round(lr_y_pred)))
print('XGBoost Accuracy:', accuracy_score(y_test, xgb_y_pred))
print('Random Forest Accuracy:', accuracy_score(y_test, rfc_y_pred))

import joblib


joblib.dump(scaler, 'scaler.joblib')


joblib.dump(svm, 'svm.joblib')
joblib.dump(lr, 'lr.joblib')
joblib.dump(xgb, 'xgb.joblib')
joblib.dump(rfc, 'rfc.joblib')

import tkinter as tk
from tkinter import messagebox
import pandas as pd
import numpy as np
import joblib


scaler = joblib.load("scaler.joblib")
svm = joblib.load("svm.joblib")
lr = joblib.load("lr.joblib")
xgb = joblib.load("xgb.joblib")
rfc = joblib.load("rfc.joblib")

def show_popup(team_id):
    
    df = pd.read_csv("/Users/prathyushachelemella/Desktop/games.csv")

    
    df.drop(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'SEASON', 'TEAM_ID_home', 'TEAM_ID_away'], axis=1,
            inplace=True)

   
    df['PTS_DIFF'] = df['PTS_home'] - df['PTS_away']

   
    df['ELO_HOME'] = 1500
    df.loc[1:, 'ELO_HOME'] = np.cumsum((df['HOME_TEAM_WINS'] - 0.5) * 20)

    
    df['ELO_AWAY'] = 1500
    df.loc[1:, 'ELO_AWAY'] = np.cumsum((1 - df['HOME_TEAM_WINS'] - 0.5) * 20)

    
    df.dropna(inplace=True)
    print(df)
 
    try:
        team_df = df[(df['HOME_TEAM_ID'] == team_id)]
    except KeyError:
        messagebox.showerror("Error", "Invalid Team ID entered!")
        return

   
    X = team_df.drop(['HOME_TEAM_WINS'], axis=1)
    y = team_df['HOME_TEAM_WINS']
    X = scaler.transform(X)
    print(X)

   
    svm_y_pred = svm.predict(X)
    lr_y_pred = lr.predict(X)
    xgb_y_pred = xgb.predict(X)
    rfc_y_pred = rfc.predict(X)



    
    svm_accuracy = accuracy_score(y, svm_y_pred)
    print(svm_accuracy)
    lr_accuracy = accuracy_score(y, np.round(lr_y_pred))
    xgb_accuracy = accuracy_score(y, xgb_y_pred)
    rfc_accuracy = accuracy_score(y, rfc_y_pred)


    message = f"Accuracy: {svm_accuracy:.2f}\n"
    message += f"Accuracy: {lr_accuracy:.2f}\n"
    message += f"Accuracy: {xgb_accuracy:.2f}\n"
    message += f"Accuracy: {rfc_accuracy:.2f}\n"
    messagebox.showinfo("Prediction Results", message)


root = tk.Tk()
root.title("NBA Game Prediction")
root.geometry("400x150")


team_id_label = tk.Label(root, text="Enter Team ID:")
team_id_label.pack()
team_id_entry = tk.Entry(root)
team_id_entry.pack()


predict_button = tk.Button(root, text="Predict", command=lambda: show_popup(int(team_id_entry.get())))
predict_button.pack()


root.mainloop()

